In [3]:
import pandas as pd
import numpy as np
import time
import heapq
import random

In [4]:
class DisasterZoneEnv:
    """
    A 2D grid environment for a drone exploring a disaster zone.

    Legend (internally stored as integers):
        0 -> Empty cell
        1 -> Obstacle
        2 -> Survivor
        3 -> Recharging Point

    'D' in a scenario array indicates the drone's starting position (only in predefined scenarios).
    """

    def __init__(self, 
                 width=8, 
                 height=8, 
                 num_obstacles=5, 
                 num_survivors=3, 
                 num_resources=2, 
                 initial_energy=20, 
                 dynamic=False, 
                 predefined_grid=None):
        """
        Initialize the environment. If a predefined grid is provided, use it; otherwise, generate randomly.

        :param width: Width of the grid.
        :param height: Height of the grid.
        :param num_obstacles: Number of obstacles.
        :param num_survivors: Number of survivors.
        :param num_resources: Number of recharging points.
        :param initial_energy: Initial energy of the drone.
        :param dynamic: Whether the environment is dynamic.
        :param predefined_grid: A predefined grid layout (optional, can contain 'D').
        """
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic
        self.energy = initial_energy
        self.dynamic_changes = 0  # Counter for dynamic changes

        # If predefined_grid is given, use that scenario
        if predefined_grid is not None:
            self.reset_with_scenario(predefined_grid)
        else:
            self.reset()

    def _get_random_empty_cell(self):
        """
        Finds a random empty cell in the grid (cell == 0).
        """
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:  # Ensure the cell is empty
                return x, y

    def reset(self):
        """
        Resets the environment to a new random configuration.
        """
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Place obstacles
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1

        # Place survivors
        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2

        # Place resources
        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3

        # Random start for the drone
        self.drone_x, self.drone_y = self._get_random_empty_cell()
        self.energy = self.initial_energy

    def reset_with_scenario(self, scenario):
        """
        Loads a predefined grid (possibly containing 'D') into the environment.
        We convert that scenario into an integer grid internally.
        """
        # Ensure scenario is at least a NumPy array
        scenario = np.array(scenario, dtype=object)  # Force an object array (handles mixed int/'D')

        # Prepare an integer grid of the same shape
        self.height, self.width = scenario.shape
        self.grid = np.zeros((self.height, self.width), dtype=int)

        drone_positions = []

        # Convert each cell from scenario into our integer grid
        for x in range(self.height):
            for y in range(self.width):
                cell_value = scenario[x, y]
                if cell_value == 'D':
                    # Mark drone position
                    drone_positions.append((x, y))
                    self.grid[x, y] = 0  # The drone is effectively on an empty cell
                else:
                    # Convert string or int to int properly
                    # If cell_value is already an int, this will do nothing special.
                    # If it's a string like '1', '2', '3', or '0', convert to int.
                    self.grid[x, y] = int(cell_value)

        if len(drone_positions) != 1:
            raise ValueError("Scenario must contain exactly one 'D' for the drone's starting position.")

        self.drone_x, self.drone_y = drone_positions[0]
        self.energy = self.initial_energy

    def apply_dynamic_changes(self, step_count):
        """
        Applies dynamic changes to the environment if self.dynamic is True.

        :param step_count: Current step count of the simulation.
        """
        if self.dynamic:
            # Example dynamic behavior:
            # 1) Add a new obstacle every 5 steps
            if step_count % 5 == 0:
                x, y = self._get_random_empty_cell()
                self.grid[x, y] = 1
                self.dynamic_changes += 1

            # 2) Move all survivors every 3 steps
            if step_count % 3 == 0:
                survivor_positions = [(x, y) for x in range(self.height) 
                                      for y in range(self.width) if self.grid[x, y] == 2]
                for (sx, sy) in survivor_positions:
                    self.grid[sx, sy] = 0  # Remove old survivor
                    new_x, new_y = self._get_random_empty_cell()
                    self.grid[new_x, new_y] = 2

    def render(self):
        """
        Renders the grid for visualization in a text-based manner.
        """
        grid_copy = self.grid.astype(str)
        grid_copy[grid_copy == '0'] = '.'
        grid_copy[grid_copy == '1'] = '#'
        grid_copy[grid_copy == '2'] = 'S'
        grid_copy[grid_copy == '3'] = 'R'

        # Mark the drone in the visualization
        grid_copy[self.drone_x, self.drone_y] = 'D'

        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")

In [10]:
import heapq

class AStarDrone:
    def __init__(self, env, initial_position, goal_position):
        # Initialize the drone with environment, starting, and goal positions
        self.env = env
        self.position = initial_position
        self.goal = goal_position
        self.visited = set()  # Track visited positions
        self.path = []  # Track the full path of positions
        self.energy = env.initial_energy  # Initial energy from the environment
        self.steps_taken = 0  # Track number of steps taken
        self.survivors_rescued = 0  # Track number of survivors rescued
        self.resources_collected = 0  # Track number of resources collected
        self.energy_used = 0  # Track total energy used

    def move(self):
        # Stop if drone reaches its goal
        if self.position == self.goal:
            print(f"Goal reached at {self.position}")
            return True

        # Stop if drone runs out of energy
        if self.energy <= 0:
            print("Out of energy!")
            return False

        # A* Search to find the next best move
        next_move = self.a_star_search(self.position, self.goal)

        if next_move:
            # Calculate the action to take and update position
            action = self.calculate_action(self.position, next_move)

            # Update the drone's position and mark it as visited
            self.position = next_move
            self.visited.add(self.position)
            self.path.append(self.position)

            # Deduct energy for each move and track energy usage
            self.energy -= 1
            self.energy_used += 1
            self.steps_taken += 1

            # Apply dynamic changes to the environment
            self.env.apply_dynamic_changes(self.steps_taken)

            # Print grid and current position
            print(f"Current Grid:")
            for row in self.env.grid:
                print(row)

            print(f"Moved to {self.position}, Current Cell: {self.env.grid[self.position[0]][self.position[1]]}")
            print(f"Energy left: {self.energy}")

            # Check grid value for survivors or resources
            current_cell = self.env.grid[self.position[0]][self.position[1]]
            print(f"Current grid value: {current_cell}")

            if current_cell == 2:  # '2' for survivors
                self.survivors_rescued += 1
                self.env.grid[self.position[0]][self.position[1]] = 0  # Mark survivor as rescued
                print(f"Survivor rescued at {self.position}")

            if current_cell == 3:  # '3' for resources
                self.resources_collected += 1
                self.env.grid[self.position[0]][self.position[1]] = 0  # Mark resource as collected
                print(f"Resource collected at {self.position}")

                # Make sure energy does not exceed the initial energy limit
                self.energy = min(self.energy + 5, self.env.initial_energy)
                print(f"Energy replenished! Current energy: {self.energy}")

            return False
        else:
            print("No valid move found.")
            return False

    def a_star_search(self, start, goal):
        """Perform A* search to find the best path from start to goal."""
        open_list = []
        heapq.heappush(open_list, (0, start))  # Priority queue (min-heap) based on f-score
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}
        visited = set()

        while open_list:
            _, current = heapq.heappop(open_list)  # Get node with the lowest f-score
            visited.add(current)

            # If the goal is reached, reconstruct the path
            if current == goal:
                return self.reconstruct_path(came_from, current)

            for neighbor in self.get_neighbors(current):
                if neighbor in visited or self.env.grid[neighbor[0], neighbor[1]] == 1:
                    continue  # Skip visited or invalid cells

                tentative_g_score = g_score[current] + 1  # Assume uniform cost for movement

                # Update scores if a better path is found
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)

                    # Add neighbor to the priority queue
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

        return None  # No valid path found

    def get_neighbors(self, position):
        """Get valid neighboring cells that are not obstacles."""
        x, y = position
        neighbors = []

        # Define directions for up, down, left, right
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # (dx, dy)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            # Ensure the position is within bounds and not an obstacle
            if 0 <= nx < self.env.height and 0 <= ny < self.env.width and self.env.grid[nx][ny] != 1:
                neighbors.append((nx, ny))

        return neighbors

    def heuristic(self, position, goal):
        """Manhattan distance as heuristic for A* search."""
        return abs(position[0] - goal[0]) + abs(position[1] - goal[1])

    def reconstruct_path(self, came_from, current):
        """Reconstruct the path from start to goal using the came_from map."""
        path = []
        while current in came_from:
            path.append(current)
            current = came_from[current]
        path.reverse()  # Reverse to get the correct order
        return path[0] if path else None  # Return the first position in the path

    def calculate_action(self, current, next_position):
        """Calculate the action based on the direction of movement."""
        dx, dy = next_position[0] - current[0], next_position[1] - current[1]

        # Define actions based on movement direction
        if dx == -1 and dy == 0:
            return 'UP'
        elif dx == 1 and dy == 0:
            return 'DOWN'
        elif dx == 0 and dy == -1:
            return 'LEFT'
        elif dx == 0 and dy == 1:
            return 'RIGHT'
        else:
            raise ValueError("Invalid move: Action not found.")

In [11]:
import time

class ScenarioTester:

    def __init__(self, agent_class, env_params):
        """
        Initialize the tester with the agent class and environment parameters.

        :param agent_class: The agent class to be tested (e.g., AStarDrone).
        :param env_params: Dictionary of environment parameters.
        """
        self.agent_class = agent_class
        self.env_params = env_params
        self.results = []  # To store results of each scenario

    def get_initial_and_goal_positions(self, grid):
        """
        Extracts the initial position (marked 'D') and a goal position (e.g., survivor or resource).
        """
        initial_position = None
        goal_position = None
        for x in range(len(grid)):
            for y in range(len(grid[0])):
                if grid[x][y] == 0:  # Assuming 0 is the empty cell and 'D' corresponds to drone
                    initial_position = (x, y)
                elif grid[x][y] == 2:  # 2 represents survivor
                    goal_position = (x, y)

        # If no goal is found, set it to a default position
        if not goal_position:
            goal_position = (len(grid) - 1, len(grid[0]) - 1)  # Default to bottom-right corner
        return initial_position, goal_position
      
    def run_scenario(self, scenario, scenario_id):
        """
        Run a predefined scenario with initial setup and agent's execution.
    
        :param scenario: A predefined grid for the environment.
        :param scenario_id: Identifier for the scenario.
        """
        # Get initial and goal positions from the scenario grid
        initial_position, goal_position = self.get_initial_and_goal_positions(scenario)

        # Create environment with the specified scenario
        env = DisasterZoneEnv(
            predefined_grid=scenario,
            initial_energy=self.env_params.get("initial_energy", 20),
            dynamic=self.env_params.get("dynamic", False)
        )

        # Initialize the agent with the initial and goal positions
        agent = self.agent_class(env, initial_position, goal_position)

        # Measure execution time
        start_time = time.time()

        print(f"Starting scenario {scenario_id} with initial position: {initial_position} and goal position: {goal_position}")
    
        step_count = 0
        while not agent.move():  # Calling move() continuously
            # Apply dynamic changes if necessary
            env.apply_dynamic_changes(step_count)
        
            # Optionally, render the environment for visualization
            env.render()

            # Track energy and position
            print(f"Agent at position {agent.position}, Energy: {env.energy}")
            print(f"Current grid value: {env.grid[agent.position[0]][agent.position[1]]}")
        
            step_count += 1

        end_time = time.time()
        computation_time = end_time - start_time

        # Record results
        self.results.append({
            "Agent Name": self.agent_class.__name__,
            "Scenario ID": scenario_id,
            "Steps Taken": getattr(agent, "steps_taken", "N/A"),
            "Survivors Rescued": getattr(agent, "survivors_rescued", "N/A"),
            "Resources Collected": getattr(agent, "resources_collected", "N/A"),
            "Energy Used": self.env_params.get("initial_energy", 20) - env.energy,
            "Computation Time (s)": computation_time
        })

        print(f"Results for {scenario_id}:")
        print(f"Survivors Rescued: {agent.survivors_rescued}")
        print(f"Resources Collected: {agent.resources_collected}")
        print(f"Energy Used: {self.env_params.get('initial_energy', 20) - env.energy}")
        print(f"Computation Time (s): {computation_time:.6f}")
        
    def display_results(self):
        """
        Display results for all tested scenarios in a formatted table.
        """
        if not self.results:
            print("No scenarios have been tested yet.")
            return

        # Display results in a tabular format
        print("=" * 100)
        print(f"{'Agent Name':<15} {'Scenario ID':<15} {'Steps Taken':<15} {'Survivors Rescued':<20} {'Resources Collected':<20} {'Energy Used':<15} {'Computation Time (s)':<20}")
        print("=" * 100)

        for result in self.results:
            print(f"{result['Agent Name']:<15} {result['Scenario ID']:<15} {result['Steps Taken']:<15} {result['Survivors Rescued']:<20} {result['Resources Collected']:<20} {result['Energy Used']:<15} {result['Computation Time (s)']:<20.6f}")
        print("=" * 100)

In [12]:
if __name__ == "__main__":
    # Example agent class (replace with your actual agent class)
    agent_class = AStarDrone  # Replace with your actual agent class
    
    # Example environment parameters (you can adjust as needed)
    env_params = {
        "initial_energy": 20,
        "dynamic": True  # Set dynamic to True for dynamic environment behavior
    }
    
    # Define different scenarios (these grids can be random or predefined)
    scenario_1 = [
        ['D', 0, 0, 0, 1],
        [0, 1, 0, 2, 0],
        [0, 0, 3, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0]
    ]

    scenario_2 = [
        ['D', 0, 1, 2],
        [0, 1, 0, 0],
        [3, 0, 1, 0],
        [0, 0, 0, 0]
    ]

    # Initialize the ScenarioTester
    tester = ScenarioTester(agent_class, env_params)

    # Run different scenarios with identifiers
    tester.run_scenario(scenario_1, "Scenario 1")
    tester.run_scenario(scenario_2, "Scenario 2")

    # Display results
    tester.display_results()

Starting scenario Scenario 1 with initial position: (4, 4) and goal position: (1, 3)
Current Grid:
[0 0 0 0 1]
[0 1 0 2 0]
[0 0 3 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
Moved to (3, 4), Current Cell: 0
Energy left: 19
Current grid value: 0
D . . . #
. # . . .
. . R . .
# . # . .
. . . # S
Energy: 20

Agent at position (3, 4), Energy: 20
Current grid value: 0
Current Grid:
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 3 0 0]
[1 0 1 0 0]
[0 0 0 1 2]
Moved to (2, 4), Current Cell: 0
Energy left: 18
Current grid value: 0
D . . . #
. # . . .
. . R . .
# . # . .
. . . # S
Energy: 20

Agent at position (2, 4), Energy: 20
Current grid value: 0
Current Grid:
[0 0 0 2 1]
[0 1 0 0 0]
[0 0 3 0 0]
[1 0 1 0 0]
[0 0 0 1 0]
Moved to (1, 4), Current Cell: 0
Energy left: 17
Current grid value: 0
D . . S #
. # . . .
. . R . .
# . # . .
. . . # .
Energy: 20

Agent at position (1, 4), Energy: 20
Current grid value: 0
Current Grid:
[0 0 0 2 1]
[0 1 0 0 0]
[0 0 3 0 0]
[1 0 1 0 0]
[0 0 0 1 0]
Moved to (1, 3), Current Cell: 0
Energy lef